In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor

from keras import backend as K

# Initialisation TPU

In [ ]:
import os

use_tpu = True

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TPU_ADDRESS = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TPU_ADDRESS = ''

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

# Chargement et correction des données

Ce dataset est utilisé pour effectuer la prédiction de la température d'une pièce en fonction de plusieurs paramètres mesurés. La fréquence originale des données est d'une minute, puis a été modifiée à 15minutes avec un filtrage. L'ensemble correspond environ à une durée de 40 jours.  
Nous allons utiliser ici la température de la chambre comme cible et sélectionner 18 séries exogènes. 

**1. Chargement des données**

In [ ]:
!rm *.txt
!curl --location --remote-header-name --remote-name "https://github.com/AlexandreBourrieau/FICHIERS/raw/main/Series_Temporelles/Multi/Data/NEW-DATA-1.T15.txt"

**2. Analyse et correction des données**

In [ ]:
# Création de la série sous Pandas
df_etude = pd.read_csv("NEW-DATA-1.T15.txt",sep=" ")
df_etude

Supprime les colonnes non utiles :
 - Date et heure
 - Exterior Entalpic 1, 2 et turbo  
   
Déplace la cible (4:Temperature_Habitacion_Sensor) en dernière colonne :

In [ ]:
df_etude = df_etude.drop(['Date','Time','19:Exterior_Entalpic_1', '20:Exterior_Entalpic_2', '21:Exterior_Entalpic_turbo'],axis=1)
cible = df_etude.pop("4:Temperature_Habitacion_Sensor")
df_etude.insert(len(df_etude.columns),"Temperature_Habitacion_Sensor",cible)
df_etude

Affiche les types :

In [ ]:
df_etude.dtypes

Modifie les type en float32 :

In [ ]:
df_etude = df_etude.astype(dtype='float32')
df_etude.dtypes

**5. Affiche les données**

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.linspace(0,len(df_etude),len(df_etude)+1),y=df_etude['Temperature_Habitacion_Sensor'], line=dict(color='blue', width=1),name="Index"))
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

# Séparation des données de test et d'entrainement

In [ ]:
# Sépare les données en entrainement et tests
pourcentage = 0.8
temps_separation = int(len(df_etude.values) * pourcentage)
date_separation = df_etude.index[temps_separation]

serie_entrainement_X = np.array(df_etude.values[:temps_separation],dtype=np.float32)
serie_test_X = np.array(df_etude.values[temps_separation:],dtype=np.float32)

print("Taille de l'entrainement : %d" %len(serie_entrainement_X))
print("Taille de la validation : %d" %len(serie_test_X))

**Normalisation des données :**

On normalise les données à l'aide de la fonction [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [ ]:
from sklearn import preprocessing

# Constrution des séries
serie_entrainement_X_norm = []
serie_test_X_norm = []

for i in range(0,len(df_etude.columns)):
  serie_entrainement_X_norm.append(serie_entrainement_X[:,i])
  serie_test_X_norm.append(serie_test_X[:,i])

serie_entrainement_X_norm = tf.convert_to_tensor(serie_entrainement_X_norm)
serie_entrainement_X_norm = tf.transpose(serie_entrainement_X_norm)
serie_test_X_norm = tf.convert_to_tensor(serie_test_X_norm)
serie_test_X_norm = tf.transpose(serie_test_X_norm)

# Initialisaton du MinMaxScaler
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(serie_entrainement_X_norm)

# Normalisation des séries
serie_entrainement_X_norm = min_max_scaler.transform(serie_entrainement_X_norm)
serie_test_X_norm = min_max_scaler.transform(serie_test_X_norm)

In [ ]:
print(serie_entrainement_X_norm.shape)
print(serie_test_X_norm.shape)

In [ ]:
# Affiche quelques séries
fig, ax = plt.subplots(constrained_layout=True, figsize=(15,5))

ax.plot(df_etude.index[:temps_separation].values,serie_entrainement_X_norm[:,0:5], label="X_Ent")
ax.plot(df_etude.index[temps_separation:].values,serie_test_X_norm[:,0:5], label="X_Val")

ax.legend()
plt.show()

# Création des datasets

In [ ]:
# Fonction permettant de créer un dataset à partir des données de la série temporelle
# X = {((X1_1,X1_2,...,X1_T),(X2_1,X2_2,...,X2_T),(X3_1,X3_2,...,X3_T)),
#       (Y1,Y2,...,YT)}
# Y = YT+1

def prepare_dataset_XY(seriesX, serieY, longueur_sequence, longueur_sortie, batch_size,shift):
  datasetX = tf.data.Dataset.from_tensor_slices(seriesX)
  datasetX = datasetX.window(longueur_sequence+longueur_sortie, shift=shift, drop_remainder=True)
  datasetX = datasetX.flat_map(lambda x: x.batch(longueur_sequence + longueur_sortie))
  datasetX = datasetX.map(lambda x: (x[0:longueur_sequence][:,:]))
  datasetX = datasetX.batch(batch_size,drop_remainder=True).prefetch(1)

  datasetY = tf.data.Dataset.from_tensor_slices(serieY)
  datasetY = datasetY.window(longueur_sequence+longueur_sortie, shift=shift, drop_remainder=True)
  datasetY = datasetY.flat_map(lambda x: x.batch(longueur_sequence + longueur_sortie))
  datasetY = datasetY.map(lambda x: (x[0:longueur_sequence][:,:]))
  datasetY = datasetY.batch(batch_size,drop_remainder=True).prefetch(1)

  datasetYPred = tf.data.Dataset.from_tensor_slices(serieY)
  datasetYPred = datasetYPred.window(longueur_sequence+longueur_sortie+1, shift=shift, drop_remainder=True)
  datasetYPred = datasetYPred.flat_map(lambda x: x.batch(longueur_sequence + longueur_sortie+1))
  datasetYPred = datasetYPred.map(lambda x: (x[0:-1][-longueur_sortie:,:]))
  datasetYPred = datasetYPred.batch(batch_size,drop_remainder=True).prefetch(1)


  dataset = tf.data.Dataset.zip((datasetX,datasetY))
  dataset = tf.data.Dataset.zip((dataset,datasetYPred))

  return dataset

In [ ]:
# Définition des caractéristiques du dataset que l'on souhaite créer
batch_size = 128
longueur_sequence = 20
longueur_sortie = 5
shift=1

# Création du dataset
dataset = prepare_dataset_XY(serie_entrainement_X_norm[:,0:-1],serie_entrainement_X_norm[:,-1:], longueur_sequence,longueur_sortie,batch_size,shift)
dataset_val = prepare_dataset_XY(serie_test_X_norm[:,0:-1],serie_test_X_norm[:,-1:],longueur_sequence,longueur_sortie,batch_size,shift)

In [ ]:
print(len(list(dataset.as_numpy_iterator())))
for element in dataset.take(1):
  print(element[0][0].shape)            # ((X1),(X2),...) = ((X1_1,X1_2,...,X1_T),(X2_1,X2_2,...,X2_T),...)
  print(element[0][1].shape)            # (Y1,Y2,...,YT)
  print(element[1].shape)               # YT+1

In [ ]:
print(len(list(dataset_val.as_numpy_iterator())))
for element in dataset_val.take(1):
  print(element[0][0].shape)            # ((X1),(X2),...) = ((X1_1,X1_2,...,X1_T),(X2_1,X2_2,...,X2_T),...)
  print(element[0][1].shape)            # Y1,Y2,...,YT
  print(element[1].shape)               # YT+1

**3. Préparation des X/Y**

In [ ]:
X1 = []
X2 = []

# Extrait les X,Y du dataset
x,y = tuple(zip(*dataset))              # x=43x((BS,10,3),(BS,9,1))
                                        # y=43x(BS,1,1)
for i in range(len(x)):
  X1.append(x[i][0])          
  X2.append(x[i][1])

X1 = tf.convert_to_tensor(X1)           # (43,BS,10,3)
X2 = tf.convert_to_tensor(X2)           # (43,BS,9,1)

X1 = np.asarray(X1,dtype=np.float32)    # (43,BS,10,3)
X2 = np.asarray(X2,dtype=np.float32)    # (43,BS,10,3)   

# Recombine les données
y = np.asarray(y,dtype=np.float32)      # 43x(BS,1,1) => (43xBS,1,1)
X1 = np.reshape(X1,(X1.shape[0]*X1.shape[1],X1.shape[2],X1.shape[3]))   # (43,BS,10,3) => (43xBS,10,3)
X2 = np.reshape(X2,(X2.shape[0]*X2.shape[1],X2.shape[2],X2.shape[3]))   # (43,BS,9,1) => (43*BS,9,1)

x_train = [X1,X2]
y_train = np.asarray(tf.reshape(y,shape=(y.shape[0]*y.shape[1],longueur_sortie,y.shape[3])))

# Affiche les formats
print(x_train[0].shape)
print(x_train[1].shape)
print(y_train.shape)


In [ ]:
X1 = []
X2 = []

# Extrait les X,Y du dataset
x,y = tuple(zip(*dataset_val))              # x=43x((BS,10,3),(BS,9,1))
                                        # y=43x(BS,1,1)
for i in range(len(x)):
  X1.append(x[i][0])          
  X2.append(x[i][1])

X1 = tf.convert_to_tensor(X1)           # (43,BS,10,3)
X2 = tf.convert_to_tensor(X2)           # (43,BS,9,1)

X1 = np.asarray(X1,dtype=np.float32)    # (43,BS,10,3)
X2 = np.asarray(X2,dtype=np.float32)    # (43,BS,10,3)   

# Recombine les données
y = np.asarray(y,dtype=np.float32)      # 43x(BS,1,1) => (43xBS,1,1)
X1 = np.reshape(X1,(X1.shape[0]*X1.shape[1],X1.shape[2],X1.shape[3]))   # (43,BS,10,3) => (43xBS,10,3)
X2 = np.reshape(X2,(X2.shape[0]*X2.shape[1],X2.shape[2],X2.shape[3]))   # (43,BS,9,1) => (43*BS,9,1)

x_val = [X1,X2]
y_val = np.asarray(tf.reshape(y,shape=(y.shape[0]*y.shape[1],longueur_sortie,y.shape[3])))

# Affiche les formats
print(x_val[0].shape)
print(x_val[1].shape)
print(y_val.shape)


# Création du modèle STAM

Le modèle STAM implanté est le suivant :

<img src="https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/VueGenerale.png?raw=true">

**1. Création de la couche de création des motifs spatiaux**

In [ ]:
class CreationMotifsSpatiaux(tf.keras.layers.Layer):
  def __init__(self, dim_motifs):
    self.dim_motifs = dim_motifs
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    couches_denses = []
    input_couches_denses = []

    # Création des N entrées des couches denses
    for i in range(input_shape[2]):
      input_couches_denses.append(tf.keras.Input(shape=(input_shape[1])))                   # input = N*(batch_size,Tin)

    # Création des N couches denses
    for i in range(input_shape[2]):
      couche_dense = tf.keras.layers.Dense(units=self.dim_motifs)(input_couches_denses[i])    # couche_dense : (batch_size,dim_motif)
      couches_denses.append(couche_dense)

    # Création de la sortie des N couches denses
    out = tf.convert_to_tensor(couches_denses)                  # (N,batch_size,dim_motif)
    out = tf.transpose(out,perm=[1,0,2])                        # (batch_size,N,dim_motif)

    # Création du modèle global
    self.Dense_Motifs = tf.keras.Model(inputs=input_couches_denses,outputs=out)
    super().build(input_shape)        # Appel de la méthode build()
    
  # Entrées :
  #     input:            Séries exogènes & cibles  : (batch_size,Tin,N)
  # Sorties :
  #     space_motifs:     Motifs spatiaux           : (batch_size,N,dim_motif)
  def call(self, input):
    input_list = tf.unstack(input,axis=2)           # N*(batch_size,Tin)
    space_motifs = self.Dense_Motifs(input_list)    # (batch_size,N,dim_motifs)
    return space_motifs

**2. Création de la couche de création des motifs temporels**

In [ ]:
class CreationMotifsTemporels(tf.keras.layers.Layer):
  def __init__(self, dim_motifs):
    self.dim_motifs = dim_motifs
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.couche_lstm1 = tf.keras.layers.LSTM(units=self.dim_motifs,return_sequences=True)
    self.couche_lstm2 = tf.keras.layers.LSTM(units=self.dim_motifs,return_sequences=True)
    super().build(input_shape)        # Appel de la méthode build()
    
  # Entrées :
  #     input:            Séries exogènes & cible  : (batch_size,Tin,N)
  # Sorties :
  #     temp_motifs:      Motifs temporels          : (batch_size,Tin,dim_motif)
  def call(self, input):
    out_lstm = self.couche_lstm1(input)            # (batch_size,Tin,dim_motif)
    out_lstm = self.couche_lstm2(out_lstm)         # (batch_size,Tin,dim_motif)
    return out_lstm

**3. Création de la couche d'attention de l'encodeur spatial**

On commence par créer la couche de calcul du score de l'encodeur spatial.  
Cette fonction est appellée par la couche d'attention spatiale à l'aide de la méthode TimeDistribued de Keras.

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/EncodeurSpatialSTAM_CalculScore2.png?raw=true'>

In [ ]:
class CalculScores_EncodeurSpatial(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()                  # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.CoucheDenseScoresEncSpatial = tf.keras.layers.Dense(units=1,activation="relu")
    super().build(input_shape)        # Appel de la méthode build()

  def compute_output_shape(self, input_shape):
    return (input_shape[0], 1)


  #     hidd_state:     hidden_state        : (batch_size,#LSTMG)
  def SetStates(self,hidd_state):
    self.hidd_state = hidd_state

   # Entrées :
  #     motifs :      Motifs spatiaux               : (batch_size,#space)
  # Sorties :
  #     score :       score                         : (batch_size,1)
  def call(self,motifs):
    motifs = tf.concat([motifs,self.hidd_state],axis=1)         # (batch_size,#space+#LSTMG)
    score = self.CoucheDenseScoresEncSpatial(motifs)      # (batch_size,1)
    return score

Puis maintenant la couche d'attention :

In [ ]:
class CalculAttention_EncodeurSpatial(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.couche_CalculScores_EncodeurSpatial = CalculScores_EncodeurSpatial()
    super().build(input_shape)        # Appel de la méthode build()


  # Entrées :
  #     input :       Motifs spatiaux               : (batch_size,N,#space)
  #     hid_state :   Hidden state décodeur spatial : (batch_size,#LSTMG)
  # Sorties :
  #     vect_contexte   Vecteur Contexte    : (batch_size,#space)
  def call(self, input,hid_state):
    # Calcul des scores
    self.couche_CalculScores_EncodeurSpatial.SetStates(hid_state)
    b = tf.keras.layers.TimeDistributed(
        self.couche_CalculScores_EncodeurSpatial)(input)                      # (batch_size,N,#space) : Timestep=N
                                                                              # (batch_size,#space) envoyé N fois en //
                                                                              # (batch_size,N,1) retourné
    # Normalisation des scores
    b = tf.keras.activations.softmax(b,axis=1)                                # (batch_size,N,1)

    # Calcul du vecteur contexte
    g = tf.multiply(input,b)        # (batch_size,N,#space)_x_(batch_size,N,1) = (batch_size,N,#space)
    g = K.sum(g,axis=1)             # (batch_size,#space)
    return g


**4. Création de la couche d'attention de l'encodeur temporel**

On commence par créer la couche de calcul du score de l'encodeur spatial.  
Cette fonction est appellée par la couche d'attention spatiale à l'aide de la méthode TimeDistribued de Keras.

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/EncodeurTemporelSTAM_CalculScore2.png?raw=true'>

In [ ]:
class CalculScores_EncodeurTemporel(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()                  # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.CoucheDenseScoresEncTemporel = tf.keras.layers.Dense(units=1,activation="relu")
    super().build(input_shape)        # Appel de la méthode build()

  def compute_output_shape(self, input_shape):
    return (input_shape[0], 1)

  #     hidd_state:     hidden_state        : (batch_size,#LSTMG)
  def SetStates(self,hidd_state):
    self.hidd_state = hidd_state

   # Entrées :
  #      motifs :      Motifs temporels               : (batch_size,#tempo)
  # Sorties :
  #     score :       score                         : (batch_size,1)
  def call(self,motifs):
    motifs = tf.concat([motifs,self.hidd_state],axis=1)         # (batch_size,#tempo+#LSTMS)
    score = self.CoucheDenseScoresEncTemporel(motifs)           # (batch_size,1)
    return score

Puis maintenant la couche d'attention :

In [ ]:
class CalculAttention_EncodeurTemporel(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.couche_CalculScores_EncodeurTemporel = CalculScores_EncodeurTemporel()
    super().build(input_shape)        # Appel de la méthode build()


  # Entrées :
  #     motifs :      Motifs temporels               : (batch_size,T,#tempo)
  #     hid_state :   Hidden state décodeur tempo    : (batch_size,#LSTMS)
  # Sorties :
  #     vect_contexte   Vecteur Contexte    : (batch_size,#tempo)
  def call(self, input,hid_state):
    # Calcul des scores
    self.couche_CalculScores_EncodeurTemporel.SetStates(hid_state)
    b = tf.keras.layers.TimeDistributed(
        self.couche_CalculScores_EncodeurTemporel)(input)                     # (batch_size,T,#tempo) : Timestep=T
                                                                              # (batch_size,#tempo) envoyé T fois en //
                                                                              # (batch_size,T,1) retourné
    # Normalisation des scores
    b = tf.keras.activations.softmax(b,axis=1)                                # (batch_size,T,1)

    # Calcul du vecteur contexte
    g = tf.multiply(input,b)        # (batch_size,T,#tempo)_x_(batch_size,T,1) = (batch_size,T,#tempo)
    g = K.sum(g,axis=1)             # (batch_size,#tempo)
    return g


**5. Création de la couche du décodeur spatial**

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/CoucheDecodeurSpatialSTAM.png?raw=true' width=800>

In [ ]:
class DecodeurSpatial(tf.keras.layers.Layer):
  def __init__(self, dim_LSTMG,dim_G,regul,drop):
    self.drop = drop
    self.regul = regul
    self.dim_LSTMG = dim_LSTMG          # Dimension des vecteurs cachés
    self.dim_G = dim_G                  # dimension réduite du vecteur contexte
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.CoucheReductionSpatiale = tf.keras.layers.Dense(units=self.dim_G,activation="relu")
    self.CoucheLSTMG = tf.keras.layers.LSTM(units=self.dim_LSTMG,return_state=True,kernel_regularizer=tf.keras.regularizers.l2(self.regul),dropout=self.drop,recurrent_dropout=self.drop,name="LSTM_DecodeurSpatial")
    super().build(input_shape)        # Appel de la méthode build()


  # Entrées :
  #     vc_spatial :      Vecteur Contexte spatial                : (batch_size,#space)
  #     hid_state_1 :     Hidden state décodeur spatial (t-1)     : (batch_size,#LSTMG)
  #     cell_state_1 :    Cell state décodeur spatial (t-1)       : (batch_size,#LSTMG)
  #     y_pred_1 :        Prédiction (t-1)                        : (batch_size,1,1)
  # Sorties :
  #     hid_state :       Hidden state décodeur spatial (t)       : (batch_size,#LSTMG)
  #     cell_state :      Cell state décodeur spatial (t)         : (batch_size,#LSTMG)

  def call(self, vc_spatial,hid_state_1,cell_state_1,y_pred_1):
    # Réduction du vecteur contexte
    rg = self.CoucheReductionSpatiale(vc_spatial)                         # (batch_size,dim_G)

    # Concaténation du vecteur réduit avec la prédictionà (t-1)
    rg = tf.concat([rg,tf.squeeze(y_pred_1,-1)],axis=1)           # (batch_size,dim_G+1)
    rg = tf.expand_dims(rg,-1)                                    # (batch_size,dim_G+1,1)

    # Applique le résultat au LSTMG
    out_dec, hid_state, cell_state = self.CoucheLSTMG(rg,initial_state=[hid_state_1,cell_state_1])         # hid_state / cell_state : (batch_size,#LSTMG)
    return hid_state,cell_state


**6. Création de la couche du décodeur temporel**

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/CoucheDecodeurTemporelSTAM.png?raw=true' width=800>

In [ ]:
class DecodeurTemporel(tf.keras.layers.Layer):
  def __init__(self, dim_LSTMS,dim_S,regul,drop):
    self.drop = drop
    self.regul = regul
    self.dim_LSTMS = dim_LSTMS          # Dimension des vecteurs cachés
    self.dim_S = dim_S                  # dimension réduite du vecteur contexte
    super().__init__()                  # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.CoucheReductionTemporelle = tf.keras.layers.Dense(units=self.dim_S,activation="relu")
    self.CoucheLSTMS = tf.keras.layers.LSTM(units=self.dim_LSTMS,return_state=True,kernel_regularizer=tf.keras.regularizers.l2(self.regul),dropout=self.drop,recurrent_dropout=self.drop,name="LSTM_DecodeurTemporel")
    super().build(input_shape)        # Appel de la méthode build()


  # Entrées :
  #     vc_tempo :        Vecteur Contexte temporel               : (batch_size,#tempo)
  #     hid_state_1 :     Hidden state décodeur tempo (t-1)       : (batch_size,#LSTMS)
  #     cell_state_1 :    Cell state décodeur tempo (t-1)         : (batch_size,#LSTMS)
  #     y_pred_1 :        Prédiction (t-1)                        : (batch_size,1,1)
  # Sorties :
  #     hid_state :       Hidden state décodeur tempo (t)         : (batch_size,#LSTMS)
  #     cell_state :      Cell state décodeur tempo (t)           : (batch_size,#LSTMS)

  def call(self, vc_tempo,hid_state_1,cell_state_1,y_pred_1):
    # Réduction du vecteur contexte
    rs = self.CoucheReductionTemporelle(vc_tempo)                 # (batch_size,dim_S)

    # Concaténation du vecteur réduit avec la prédictionà (t-1)
    rs = tf.concat([rs,tf.squeeze(y_pred_1,-1)],axis=1)           # (batch_size,dim_S+1)
    rs = tf.expand_dims(rs,-1)                                    # (batch_size,dimg_S+1,1)

    # Applique le résultat au LSTMG
    out_dec, hid_state, cell_state = self.CoucheLSTMS(rs,initial_state=[hid_state_1,cell_state_1])         # hid_state / cell_state : (batch_size,#LSTMS)
    return hid_state,cell_state


**4. Création de la couche réseau**

<img src='https://github.com/AlexandreBourrieau/FICHIERS/blob/main/Series_Temporelles/Multi/images/ReseauSTAM2.png?raw=true'>

In [ ]:
class Net_STAM(tf.keras.layers.Layer):
  def __init__(self,creationmotifsspatiaux, creationmotifstemporels, encodeurspatial,encodeurtemporel, decodeurspatial,decodeurtemporel,longueur_sequence, longueur_sortie, regul, drop,dim_LSTMG,dim_LSTMS):
    self.creationmotifsspatiaux = creationmotifsspatiaux
    self.creationmotifstemporels = creationmotifstemporels
    self.encodeurspatial = encodeurspatial
    self.encodeurtemporel = encodeurtemporel
    self.decodeurspatial = decodeurspatial
    self.decodeurtemporel = decodeurtemporel
    self.longueur_sequence = longueur_sequence
    self.longueur_sortie = longueur_sortie
    self.regul = regul
    self.drop = drop
    self.dim_LSTMG = dim_LSTMG
    self.dim_LSTMS = dim_LSTMS
    super().__init__()                # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.CoucheConcatenation = tf.keras.layers.Dense(units=self.dim_LSTMG+self.dim_LSTMS,activation="tanh")
    self.CoucheGenerateur = tf.keras.layers.Dense(units=1)
    super().build(input_shape)        # Appel de la méthode build()

  # Entrées :
  #     input:          Series exogènes     : (batch_size,Tin,#dim)
  #     cible:          Cible               : (batch_size,Tin,1)
  # Sorties :
  #     sortie:         Prédiction Y        : (batch_size,longueur_sortie,1)
  def call(self,input,cible):
    # Création des motifs spatiaux et temporels
    motifs_spatiaux = self.creationmotifsspatiaux(tf.concat([input,cible],axis=2))      # (batch_size,N,#space) N=#dim+1
    motifs_temporels = self.creationmotifstemporels(tf.concat([input,cible],axis=2))    # (batch_size,N,#tempo)

    # Initialise les hidden_states et les cells_states
    hid_state_spatial_1 = tf.matmul(input[:,0:1,0:1],tf.zeros(shape=(1,self.dim_LSTMG)))    # (batch_size,1,1)x(1,#LSTMG) = (batch_size,1,#LSTMG)
    hid_state_spatial_1 = tf.transpose(hid_state_spatial_1,perm=[0,2,1])                    # (batch_size,#LSTMG,1)
    hid_state_spatial_1 = tf.squeeze(hid_state_spatial_1,-1)                                # (batch_size,1)
    cell_state_spatial_1 = hid_state_spatial_1

    hid_state_temporel_1 = tf.matmul(input[:,0:1,0:1],tf.zeros(shape=(1,self.dim_LSTMS)))    # (batch_size,1,1)x(1,#LSTMS) = (batch_size,1,#LSTMS)
    hid_state_temporel_1 = tf.transpose(hid_state_temporel_1,perm=[0,2,1])                   # (batch_size,#LSTMS,1)
    hid_state_temporel_1 = tf.squeeze(hid_state_temporel_1,-1)                               # (batch_size,1)
    cell_state_temporel_1 = hid_state_temporel_1

    # Initialise la prédiction à t=0
    y_pred_1 = tf.matmul(cible[:,0:1,:],tf.zeros(shape=(1,1)))    # (batch_size,1,1)x(1,1) = (batch_size,1,1)

    sorties = []

    for t in range(self.longueur_sortie):
      # Calcul des vecteurs contextes spatiaux et temporels
      g = self.encodeurspatial(motifs_spatiaux,hid_state_spatial_1)                               # (batch_size,#space)
      s = self.encodeurtemporel(motifs_temporels,hid_state_temporel_1)                            # (batch_size,#tempo)

      # Décodeur spatial & temporel
      hid_state_dec_spatial,cell_state_dec_spatial = self.decodeurspatial(                        # (batch_size,#LSTMG)
          g,hid_state_spatial_1,cell_state_spatial_1,y_pred_1)
      hid_state_dec_temporel,cell_state_dec_temporel = self.decodeurtemporel(                     # (batch_size,#LSTMS)
          s,hid_state_temporel_1,cell_state_temporel_1,y_pred_1)
      
      # Concaténation des hidden states et des vecteurs contextes
      out_dec = tf.concat([hid_state_dec_spatial,hid_state_dec_temporel,g,s],axis=1)              # (batch_size,#LSTMG+LSTMS+#space+#tempo)

      y_pred_1 = self.CoucheConcatenation(out_dec)                                                # (batch_size,#LSTMG+#LSTMS)
      y_pred_1 = self.CoucheGenerateur(y_pred_1)                                                 # (batch_size,1)
      y_pred_1 = tf.expand_dims(y_pred_1,-1)                                                      # (batch_size,1,1)

      sorties.append(y_pred_1)
      hid_state_spatial_1 = hid_state_dec_spatial
      cell_state_spatial_1 = cell_state_dec_spatial
      hid_state_temporel_1 = hid_state_dec_temporel
      cell_state_temporel_1 = cell_state_dec_temporel

    sorties = tf.convert_to_tensor(sorties)         # (longueur_sortie,batch_size,1,1)
    sorties = tf.squeeze(sorties,-1)                # (longueur_sortie,batch_size,1)
    sorties = tf.transpose(sorties,perm=[1,0,2])    # (batch_size,longueur_sortie,1)

    return sorties

**4. Création du modèle**

In [ ]:
dim_motifs_spatiaux = 128
dim_motifs_temporels = 128

dim_LSTMG = 128
dim_G = 128
dim_LSTMS = 128
dim_S = 128
drop=0.0
l2reg=0.0

def get_model():
  entrees_exo = tf.keras.layers.Input(shape=(longueur_sequence,x_train[0].shape[2]))
  entrees_cible = tf.keras.layers.Input(shape=(longueur_sequence,1))

  creationmotifsspatiaux = CreationMotifsSpatiaux(dim_motifs=dim_motifs_spatiaux)
  creationmotifstemporels = CreationMotifsTemporels(dim_motifs=dim_motifs_temporels)
  encodeurspatial = CalculAttention_EncodeurSpatial()
  encodeurtemporel = CalculAttention_EncodeurTemporel()
  decodeurspatial = DecodeurSpatial(dim_LSTMG=dim_LSTMG,dim_G=dim_G,regul=l2reg,drop=drop)
  decodeurtemporel = DecodeurTemporel(dim_LSTMS=dim_LSTMS,dim_S=dim_S,regul=l2reg,drop=drop)


  sortie = Net_STAM(creationmotifsspatiaux,creationmotifstemporels,encodeurspatial,encodeurtemporel,decodeurspatial,decodeurtemporel,longueur_sequence,longueur_sortie,l2reg,drop,dim_LSTMG,dim_LSTMS)(entrees_exo,entrees_cible)

  model = tf.keras.Model([entrees_exo,entrees_cible],sortie)
  return model

# Entrainement avec TPU

In [ ]:
from google.colab import files

max_periodes = 500

strategy = tf.distribute.TPUStrategy(resolver)
with strategy.scope():
  # Création du modèle
  model = get_model()

  # Définition des paramètres liés à l'évolution du taux d'apprentissage
  lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
      initial_learning_rate=0.001,
      decay_steps=50,
      decay_rate=0.01)

  optimiseur=tf.keras.optimizers.Adam(learning_rate=lr_schedule)

  # Utilisation de la méthode ModelCheckPoint
  CheckPoint = tf.keras.callbacks.ModelCheckpoint("poids_train.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only = True, mode='auto', save_freq='epoch')

  # Compile le modèle
  model.compile(loss="mse", optimizer=optimiseur)

  # Entraine le modèle
  historique = model.fit(x=[x_train[0],x_train[1]],y=y_train,validation_data=([x_val[0],x_val[1]],y_val), epochs=max_periodes,verbose=1, callbacks=[CheckPoint,tf.keras.callbacks.EarlyStopping(monitor='loss', patience=60)],batch_size=batch_size)

files.download('poids_train.hdf5')

In [ ]:
model.load_weights("poids_train.hdf5")

In [ ]:
erreur_entrainement = historique.history["loss"]
erreur_validation = historique.history["val_loss"]

# Affiche l'erreur en fonction de la période
plt.figure(figsize=(10, 6))
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_entrainement, label="Erreurs sur les entrainements")
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_validation, label ="Erreurs sur les validations")
plt.legend()

plt.title("Evolution de l'erreur en fonction de la période")

In [ ]:
start = 400

erreur_entrainement = historique.history["loss"]
erreur_validation = historique.history["val_loss"]

# Affiche l'erreur en fonction de la période
plt.figure(figsize=(10, 6))
plt.plot(np.arange(0,len(erreur_entrainement[start:])),erreur_entrainement[start:], label="Erreurs sur les entrainements")
plt.plot(np.arange(0,len(erreur_entrainement[start:])),erreur_validation[start:], label ="Erreurs sur les validations")
plt.legend()

plt.title("Evolution de l'erreur en fonction de la période")

In [ ]:
model.evaluate(x=[x_train[0],x_train[1]],y=y_train)
model.evaluate(x=[x_val[0],x_val[1]],y=y_val)

# Chargement du modèle pré-entrainé

**STAM_SML2010-MultiStep**  

  - Vecteurs LSTM : 128  
  - Longueur entrée : 20  
  - Longueur sortie : 5
  - Drop : 0.0
  - L2 : 0.00  
  - Batch Size : 128  
  - Périodes : 500   


In [ ]:
model = get_model()

In [ ]:
!rm *.hdf5
!curl --location --remote-header-name --remote-name "https://github.com/AlexandreBourrieau/FICHIERS/raw/main/Series_Temporelles/Multi/Models/Multi_STAM_SML2010-Multistep_L20.hdf5"

In [ ]:
model.load_weights("Multi_STAM_SML2010-Multistep_L20.hdf5")

# Prédictions Multistep

In [ ]:
pred_ent = model.predict([x_train[0],x_train[1]],verbose=1)
pred_val = model.predict([x_val[0],x_val[1]],verbose=1)

In [ ]:
import plotly.graph_objects as go

decalage = longueur_sortie

fig = go.Figure()

# Affiche les courbes originales
fig.add_trace(go.Scatter(x=df_etude.index,y=tf.squeeze(serie_entrainement_X_norm[:,-1:],-1),line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation:],y=tf.squeeze(serie_test_X_norm[:,-1:],-1),line=dict(color='red', width=1)))

#Calcul les prédictions sur l'entrainement
pred = []
pred_index = []
step_time = []
step_val = []

max = int(len(pred_ent)/longueur_sortie)
#max = 10
for i in range(0,max):
  pred.append(tf.squeeze(pred_ent[i*longueur_sortie,0:decalage,:],1))
  pred_index.append(df_etude.index[longueur_sequence+i*longueur_sortie:longueur_sequence+(i+1)*longueur_sortie])
  step_val.append(pred_ent[i*longueur_sortie,0,0])
  step_time.append(df_etude.index[longueur_sequence+i*longueur_sortie])

pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

pred_index = np.asarray(pred_index)
pred_index = np.reshape(pred_index,(pred_index.shape[0]*pred_index.shape[1]))

fig.add_trace(go.Scatter(x=pred_index,y=pred, mode='lines', line=dict(color='green', width=1)))
fig.add_trace(go.Scatter(x=step_time,y=step_val, mode='markers', line=dict(color='black', width=1)))

#Calcul les prédictions sur les validations
pred = []
pred_index = []
step_time = []
step_val = []
max = int(len(pred_val)/longueur_sortie)
#max = 10
for i in range(0,max):
  pred.append(tf.squeeze(pred_val[i*longueur_sortie,0:decalage,:],1))
  pred_index.append(df_etude.index[temps_separation+i*decalage+longueur_sequence:temps_separation+i*decalage+longueur_sequence+longueur_sortie])
  step_val.append(pred_val[i*longueur_sortie,0,0])
  step_time.append(df_etude.index[temps_separation+i*decalage+longueur_sequence])

pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))
pred_index = np.asarray(pred_index)
pred_index = np.reshape(pred_index,(pred_index.shape[0]*pred_index.shape[1]))

fig.add_trace(go.Scatter(x=pred_index,y=pred, mode='lines', line=dict(color='green', width=1)))
fig.add_trace(go.Scatter(x=step_time,y=step_val, mode='markers', line=dict(color='black', width=1)))

# Affiche les prédictions
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**Erreurs en multi step**

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

decalage = longueur_sortie

#Calcul les prédictions sur l'entrainement
pred = []
pred_index = []
step_time = []
step_val = []

max = int(len(pred_ent)/longueur_sortie)
for i in range(0,max):
  pred.append(tf.squeeze(pred_ent[i*longueur_sortie,0:decalage,:],1))

pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

fig.add_trace(go.Scatter(x=df_etude.index[longueur_sequence:],y=serie_entrainement_X_norm[longueur_sequence:-(serie_entrainement_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]),-1],line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=df_etude.index[longueur_sequence:],y=pred,line=dict(color='green', width=1)))


fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

mse_ent = tf.keras.losses.mse(serie_entrainement_X_norm[longueur_sequence:-(serie_entrainement_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]),-1],pred)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

#Calcul les prédictions sur les validations
pred = []
max = int(len(pred_val)/longueur_sortie)
for i in range(0,max):
  pred.append(tf.squeeze(pred_val[i*longueur_sortie,0:decalage,:],1))

pred = tf.convert_to_tensor(pred).numpy()
pred = np.reshape(pred,(pred.shape[0]*pred.shape[1]))

fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence::],y=serie_test_X_norm[longueur_sequence:-(serie_test_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]),-1],line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=df_etude.index[temps_separation+longueur_sequence::],y=pred,line=dict(color='green', width=1)))


fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

mse_test = tf.keras.losses.mse(serie_test_X_norm[longueur_sequence:-(serie_test_X_norm[longueur_sequence:,:].shape[0]-pred.shape[0]),-1],pred)

In [ ]:
print(mse_ent)
print(mse_test)